In [ ]:
%cd ..
%load_ext autoreload
%autoreload 2

In [ ]:
import random

# LOCAL imports
import emv
import emv.utils
import emv.io.media
import emv.features.text
import emv.llm.retriever
import emv.db.queries


from emv.db.dao import DataAccessObject
DataAccessObject().set_user_id(3)

LOG = emv.utils.get_logger()

In [22]:
import litellm
from litellm import completion

litellm.set_verbose=False

response = completion(
            model="ollama/mixtral:8x7b-instruct-v0.1-fp16", 
            messages = [{ "content": "Tell me about Kirell Benzi","role": "user"}], 
            api_base="http://192.168.1.42:11434"
)

response.choices[0].message.content

16:27:55 - LiteLLM:INFO: Wrapper: Completed Call, calling success_handler


INFO:LiteLLM:Wrapper: Completed Call, calling success_handler


' Kirell Benzi is a renowned data artist and researcher known for his innovative work in the field of data visualization. He was born on March 3, 1990, in France. Benzi has a background in both art and science, holding a degree in Applied Mathematics from the University of Paris-Sud and a Master\'s degree in Interactive Design and Digital Animation from the Ensci Les Ateliers.\n\nBenzi gained significant attention in 2015 when he created "The Wave," a mesmerizing data visualization of the world\'s stock exchanges, which was later featured in The New Yorker. In this project, Benzi employed a novel technique called "Flow Maps" to illustrate complex financial data in an engaging and intuitive manner.\n\nIn addition to his work as a data artist, Benzi is also a researcher at the Swiss Data Science Center, where he focuses on developing new methods for visualizing and interpreting large datasets. He has published numerous papers on these topics and frequently shares his insights through tal

In [28]:
from guidance import models, gen, select
llm = models.LiteLLMCompletion(api_base="http://192.168.1.42:11434", model='ollama/mixtral:8x7b-instruct-v0.1-fp16') 

# append text or generations to the model
llm + f'''\
Do you want a joke or a poem? A {select(['joke', 'poem'])}.
Okay, here is a one-liner: "{gen(stop='"')}"
'''

In [ ]:
retriever = emv.llm.retriever.DocumentRetriever()
retriever.fetch_similar("La Stratégie Energétique 2050 en Suisse a, entre autres, pour objectif d’accroître drastiquement le nombre d’installations solaires photovoltaïques. Au moins 35 TWh d'électricité devront être produits en 2035 grâce aux énergies renouvelables, sans compter l’hydraulique, et 45 TWh en 2050. Comment atteindre ces objectifs d’une manière efficace et équitable pour chaque ville et/ou commune suisse ?", 5)

In [ ]:
from llama_index.llms import Ollama
from llama_index.llms import ChatMessage
import textwrap

In [ ]:
llm = Ollama(base_url="http://192.168.1.42:11434", model="mistral", request_timeout=30.0)

In [ ]:
messages = [
    ChatMessage(
        role="system", content="You are a developer with a colorful personality"
    ),
    ChatMessage(role="user", content="What is my name"),
]
resp = llm.stream_chat(messages)

for r in resp:
    print(r.delta, end="")

In [ ]:
from sqlalchemy import make_url
from dotenv import load_dotenv
import os
import psycopg2

load_dotenv()  # Load environment variables from .env file

# Get the environment variables
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')

# Construct the connection string
connection_string = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
url = make_url(connection_string)

vector_store = PGVectorStore.from_params(
    database=db_name,
    host=url.host,
    password=url.password,
    port=url.port,
    user=url.username,
    table_name="feature",
    embed_dim=1024,  # openai embedding dimension
)

# storage_context = StorageContext.from_defaults(vector_store=vector_store)
# index = VectorStoreIndex.from_documents(
#     documents, storage_context=storage_context, show_progress=True
# )
# query_engine = index.as_query_engine()

In [ ]:
# from llama_index.core.bridge.pydantic import PrivateAttr
# from llama_index.core.embeddings import BaseEmbedding

# class LlamaIndexEmbeddings(BaseEmbedding):
#     _model: emv.features.text.TextEmbedder = PrivateAttr()
#     def __init__(
#         self,
#         **kwargs: Any,
#     ) -> None:
#         self._model = emv.features.text.TextEmbedder()
#         super().__init__(**kwargs)

#     @classmethod
#     def class_name(cls) -> str:
#         return "LlamaIndexEmbeddings"

#     async def _aget_query_embedding(self, query: str) -> List[float]:
#         return self._get_query_embedding(query)

#     async def _aget_text_embedding(self, text: str) -> List[float]:
#         return self._get_text_embedding(text)

#     def _get_query_embedding(self, query: str) -> List[float]:
#         embeddings = self._model.encode(query)
#         return embeddings[0]

#     def _get_text_embedding(self, text: str) -> List[float]:
#         embeddings = self._model.encode(text)
#         return embeddings[0]

#     def _get_text_embeddings(self, texts: List[str]) -> List[List[float]]:
#         embeddings = self._model.encode(texts)
#         return embeddings